In [1]:
import os 
import pickle 
import torch 
from tqdm.auto import tqdm 

with open(os.path.expanduser('~/dataset/OAG/OAG-CS/hg_full.dict.pkl'), 'rb') as fp: 
    graph_info_dict = pickle.load(fp) 
    
PF_edge_index = graph_info_dict['edge_index_dict'][('paper', 'PF_L2', 'field')]
paper_year_vec = graph_info_dict['paper_year'] 
num_paper_nodes = len(paper_year_vec) 

num_paper_nodes, PF_edge_index.shape, paper_year_vec.shape 

(546704, torch.Size([2, 2347239]), torch.Size([546704]))

In [2]:
field_id_map: dict[int, int] = dict() 

for P_nid, F_nid in zip(*PF_edge_index.tolist()): 
    if F_nid not in field_id_map: 
        field_id_map[F_nid] = len(field_id_map)
        
num_field_nodes = len(field_id_map) 

num_field_nodes

17750

In [3]:
field_id_map: dict[int, int] = dict() 

for P_nid, F_nid in zip(*PF_edge_index.tolist()): 
    if F_nid not in field_id_map: 
        field_id_map[F_nid] = len(field_id_map)
        
paper_label_mat = torch.zeros(num_paper_nodes, num_field_nodes, dtype=torch.bool) 
paper_pretrain_mask = torch.zeros(num_paper_nodes, dtype=torch.bool)
paper_train_mask = torch.zeros(num_paper_nodes, dtype=torch.bool)
paper_val_mask = torch.zeros(num_paper_nodes, dtype=torch.bool)
paper_test_mask = torch.zeros(num_paper_nodes, dtype=torch.bool)

for P_nid, F_nid in tqdm(zip(*PF_edge_index.tolist())): 
    P_year = paper_year_vec[P_nid] 
    paper_label_mat[P_nid, field_id_map[F_nid]] = True 
    
    if P_year < 2014:
        paper_pretrain_mask[P_nid] = True 
    elif P_year >= 2014 and P_year <= 2016: 
        paper_train_mask[P_nid] = True 
    elif P_year == 2017: 
        paper_val_mask[P_nid] = True 
    else:
        paper_test_mask[P_nid] = True 

paper_label_mat.shape, \
    paper_label_mat.float().mean(), \
    paper_pretrain_mask.sum(), \
    paper_train_mask.sum(), \
    paper_val_mask.sum(), \
    paper_test_mask.sum(), \
    paper_pretrain_mask.float().mean(), \
    paper_train_mask.float().mean(), \
    paper_val_mask.float().mean(), \
    paper_test_mask.float().mean()

0it [00:00, ?it/s]

(torch.Size([546704, 17750]),
 tensor(0.0002),
 tensor(329924),
 tensor(111086),
 tensor(38397),
 tensor(50401),
 tensor(0.6035),
 tensor(0.2032),
 tensor(0.0702),
 tensor(0.0922))

In [4]:
graph_info_dict['paper_label'] = paper_label_mat 
graph_info_dict['paper_pretrain_mask'] = paper_pretrain_mask 
graph_info_dict['paper_train_mask'] = paper_train_mask 
graph_info_dict['paper_val_mask'] = paper_val_mask 
graph_info_dict['paper_test_mask'] = paper_test_mask 

with open(os.path.expanduser('~/dataset/OAG/OAG-CS/hg_field.dict.pkl'), 'wb') as fp: 
    pickle.dump(graph_info_dict, fp) 